## [範例重點]
了解 sklearn 中，GridSearchCV 的使用方法與原理

In [1]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
# 讀取波士頓房價資料集
boston = datasets.load_boston()

# 切分訓練集/測試集
x_train, x_test, y_train, y_test = train_test_split(boston.data, boston.target, test_size=0.25, random_state=42)

# 建立模型
clf = GradientBoostingRegressor(random_state=7)

In [3]:
# 先看看使用預設參數得到的結果，約為 8.379 的 MSE
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(metrics.mean_squared_error(y_test, y_pred))

8.913775994322064


In [4]:
# 設定要訓練的超參數組合
n_estimators = [100, 200, 300]
max_depth = [1, 3, 5]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(clf, param_grid, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

# 預設會跑 3-fold cross-validadtion，總共 9 種參數組合，總共要 train 27 次模型

D:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    1.7s finished
D:\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [5]:
# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best Accuracy: -13.118704 using {'max_depth': 3, 'n_estimators': 200}


In [6]:
grid_result.best_params_

{'max_depth': 3, 'n_estimators': 200}

In [7]:
# 使用最佳參數重新建立模型
clf_bestparam = GradientBoostingRegressor(max_depth=grid_result.best_params_['max_depth'],
                                           n_estimators=grid_result.best_params_['n_estimators'])

# 訓練模型
clf_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = clf_bestparam.predict(x_test)

In [8]:
# 調整參數後約可降至 8.30 的 MSE
print(metrics.mean_squared_error(y_test, y_pred))

8.363411253767895


[作業重點]
了解如何使用 Sklearn 中的 hyper-parameter search 找出最佳的超參數

[作業]
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [9]:
# 使用葡萄酒的資料
wine = datasets.load_wine()

In [10]:
# 觀察該資料集,共有13個 feature
X = wine .data
print("Data shape: ", X.shape)

Data shape:  (178, 13)


In [11]:
# 使用 pandas 套件將資料欄位印出
import pandas as pd
df_X = pd.DataFrame(X)
df_X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0


In [12]:
print(wine.feature_names)

['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline']


In [13]:
# 切分訓練集/測試集
x_train, x_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.25, random_state=4)

In [14]:
# 建立 GradientBoostingRegressor 模型

## loss : {‘ls’, ‘lad’, ‘huber’, ‘quantile’}, optional (default=’ls’)
# 'ls'指的是最小二乘回歸。
# 'lad'（最小絕對偏差）是一種高度魯棒的損失函數，僅基於輸入變量的順序信息。
# 'huber'是兩者的結合。
# 'quantile'允許分位數回歸（用於alpha指定分位數）。

## learning_rate : float, optional (default=0.1)
# 學習率會縮小每棵樹的貢獻 learning_rate。在 learning_rate和 n_estimators之間進行權衡。

## n_estimators : int (default=100)
# 要執行的助推階段的數量。對於過度擬合，梯度增強相當穩健，因此大量通常會產生更好的性能。

## subsample ： float，optional（默認值= 1.0）
# 用於擬合各個基礎學習者的樣本分數。
# 如果小於 1.0，則會導致隨機梯度提升。
# subsample與參數 n_estimators 交互選擇導致減少差異和增加偏差。subsample < 1.0

## criterion : string, optional (default=”friedman_mse”)
# 衡量分裂質量的功能。支持的標準是弗里德曼的改進得分的均方誤差 “friedman_mse”，均方誤差的 “mse” 和平均絕對誤差的 “mae”。
# 預設值 “friedman_mse”通常是最好的，因為它可以在某些情況下提供更好的近似值。

## min_samples_split : int, float, optional (default=2)
# 拆分內部節點所需的最小樣本數：
# 如果是 int，則考慮 min_samples_split為最小數量。
# 如果是浮點數，那麼它 min_samples_split是一個分數，是每個分割的最小樣本數。ceil(min_samples_split * n_samples)

## min_samples_leaf : int, float, optional (default=1)
# 葉節點所需的最小樣本數。
# 只有 min_samples_leaf在每個左右分支中留下至少訓練樣本時，才會考慮任何深度的分裂點。這可能具有平滑模型的效果，尤其是在回歸中。
# 如果是 int，則考慮 min_samples_leaf為最小數量。
# 如果是 float，那麼它 min_samples_leaf是一個分數，是每個節點的最小樣本數。ceil(min_samples_leaf * n_samples)

## min_weight_fraction_leaf : float, optional (default=0.)
# 需要在葉節點處的權重總和（所有輸入樣本的總和）的最小加權分數。當未提供 sample_weight時，樣本具有相同的權重。

## max_depth : integer, optional (default=3)
# 個體回歸估計量的最大深度。最大深度限制樹中的節點數。調整此參數以獲得最佳性能; 最佳值取決於輸入變量的相互作用。

## min_impurity_decrease : float, optional (default=0.)
# 如果該分裂導致雜質的減少大於或等於該值，則將分裂節點。

## min_impurity_split : float, (default=1e-7)
# 樹木生長早期停止的門檻。如果節點的雜質高於閾值，節點將分裂，否則它是葉子。

## init : estimator or ‘zero’, optional (default=None)
# 用於計算初始預測的估算器對象。 init必須提供 fit和 predict。
# 如果為 “零 ”，則初始原始預測設置為零。預設使用 a DummyEstimator，預測平均目標值（對於 loss ='ls'）或其他損失的分位數。

## random_state : int, RandomState instance or None, optional (default=None)
# 如果是 int，則 random_state是隨機數生成器使用的種子; 如果是 instance，則 random_state是隨機數生成器; 
# 如果為 None，則隨機數生成器是由其使用的 RandomState instance np.random。

## max_features : int, float, string or None, optional (default = None) 
# 尋找最佳分割時要考慮的功能數量

## alpha : float (default=0.9)
# huber損失函數和分位數損失函數的 α分位數。只有當 loss='huber'或 loss='quantile'。

## verbose : int, default: 0
# 啟用詳細輸出。如果為1則會偶爾打印進度和性能（樹越多，頻率越低）。如果大於1，則它會打印每棵樹的進度和性能。

## max_leaf_nodes : int or None, optional (default=None)
# max_leaf_nodes以最好的方式種植樹木。最佳節點定義為雜質的相對減少。如果為 None則無限數量的葉節點。

## warm_start : bool, default: False
# 設置 True為時，重用上一個調用的解決方案以適合併向集合中添加更多估計器，否則，只需擦除以前的解決方案。

## presort : bool or ‘auto’, optional (default=’auto’)
# 是否預先分配數據以加快擬合中最佳分割的發現。
# 預設情況下，自動模式將使用密集數據的預分類，並預設對稀疏數據進行常規排序。在稀疏數據上將presort設置為true將引發錯誤。

## validation_fraction : float, optional, default 0.1
# 將訓練數據的比例留作早期停止的驗證集。必須介於 0和 1之間。僅在 n_iter_no_change設置為整數時使用。

## n_iter_no_change : int, default None
# n_iter_no_change用於確定在驗證分數沒有改善時是否將使用提前停止來終止培訓。
# 預設情況下，它設置為 “無”以禁用提前停止。
# 如果設置為數字，它將保留 validation_fraction訓練數據的大小作為驗證，
# 並在驗證得分在所有先前 n_iter_no_change的迭代次數中沒有改善時終止訓練。

## tol : float, optional, default 1e-4
# 容忍早期停止。如果損失沒有至少改善 tol的 n_iter_no_change迭代次數（如果設置為數字），則訓練停止。

clf = GradientBoostingRegressor(random_state=7)

In [15]:
# 先看看使用參數 random_state=7得到的結果，約為 0.03175 的 MSE
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(metrics.mean_squared_error(y_test, y_pred))

0.031755142740968266


In [16]:
# 設定要訓練的超參數組合
n_estimators = [100, 200, 300]
max_depth = [1, 3, 5]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
# 1.estimator：所使用的分類器，如 estimator=RandomForestClassifier(), 並且傳入除需要確定最佳的參數之外的其他參數。
#             每一個分類器都需要一個 scoring參數，或者score方法。
# 2.param_grid：值為字典或者列表，即需要最優化的參數的取值，param_grid = param_test1，param_test1 = {'n_estimators':range(10,71,10)}。
# 3.scoring :準確度評價標準，預設None,這時需要使用score函数；或者如scoring='roc_auc'，
#            根據所選模型不同，評價準則不同。字符串（函数名），或是可调用對象，需要其函數名型如：scorer(estimator, X, y)；
#            如果是None，則使用estimator的誤差估計函數。
# 4.cv :交叉驗證參數，預設None，使用三折交叉驗證。指定fold數量，預設為3，也可以是 yield訓練/測試數據的生成器。
# 5.refit :預設為True,程序將會以交叉驗證訊練集得到的最佳參數，重新對所有可用的訓練集與開發集進行，
#          作為最终用於性能評估的最佳模型參數。即在搜索參數结束後，用最佳參數結果再次 fit一遍全部數據集。
# 6.iid:預設 True,為True時，預設為各個樣本fold機率分布一致，誤差估計為所有樣本之和，而非各個fold的平均。
# 7.verbose：0：不输出訓練過程，1：偶爾输出，>1：對每個子模型都输出。
# 8.n_jobs: 平行處理數，int：個数,-1：跟 CPU核數一致, 1:預設值。
# 9.pre_dispatch：指定總共分發的並行任務數。當 n_jobs大於1時，數據將在每個運行點進行複製，
#                 這可能導致 OOM，而設置 pre_dispatch參數，則可以预先劃分總共的job數量，使數據最多被複製 pre_dispatch次

grid_search = GridSearchCV(clf, param_grid, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

# 預設會跑 3-fold cross-validadtion，總共 9 種參數組合，總共要 train 27 次模型

D:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  27 | elapsed:    0.0s remaining:    0.0s


Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    0.2s finished
D:\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [17]:
# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best Accuracy: -0.037197 using {'max_depth': 3, 'n_estimators': 100}


In [18]:
grid_result.best_params_

{'max_depth': 3, 'n_estimators': 100}

In [19]:
# 使用最佳參數重新建立模型
clf_bestparam = GradientBoostingRegressor(max_depth=grid_result.best_params_['max_depth'],
                                           n_estimators=grid_result.best_params_['n_estimators'])

# 訓練模型
clf_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = clf_bestparam.predict(x_test)

In [20]:
# 調整參數後約可降至 0.02590 的 MSE
print(metrics.mean_squared_error(y_test, y_pred))

0.025904492023357623
